In [ ]:
api_key = None

In [ ]:
import ollama
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel
from sklearn.metrics import confusion_matrix
import numpy as np
from typing import List
from openai import OpenAI
from typing import Literal
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
from matplotlib.colors import ListedColormap

import tiktoken
from functools import partial


def ollamaResponse(content):
    response = ollama.chat(model='llama3', messages=[
    {
        'role': 'user',
        'content': content,
    },
    ])
    return response['message']['content']

def openAIReponse(content, model = 'gpt-3.5-turbo'):
    client = OpenAI(api_key=api_key)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": content}
        ]
    )
    return completion.choices[0].message.content


class Response(BaseModel):
    response: bool
    
class Classification(BaseModel):
    response: str# Literal['context', 'uses', 'extends']

class Citation(BaseModel):
    citation: str# Literal['context', 'uses', 'extends']
    
    
def mask_apply(series, mask, function, fill_none = True):
    mask = mask if mask is not None else [True for _ in range(len(series))]
    result = []
    for ind, element in tqdm(zip(mask, series), total = len(mask)):
        result.append(function(element) if ind else element if fill_none else None)
    return result




In [ ]:
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/uniform_sample/results/9-13-and-8-28-with-confidence.csv'
df = pd.read_csv(path)

print(len(df))


context_fp = ((df['classification'] == 'uses') | (df['classification'] == 'extends')) & (df['alex'] == 'context')
misattribution_mask = df['misattribution'].notna()
ignore_mask = (df['multisentence'].apply(lambda s: s.find('\\begin{tab') != -1)) | (df['strippedModelKey'] == 'ase')
ambiguity_mask = df['alex_confidence'] <= 5

print(f"False positives: {(context_fp & misattribution_mask & ~ignore_mask).sum()}")
assert((context_fp & misattribution_mask).sum() == context_fp.sum()), f"Missing misattribution label for some false positives"
(context_fp & (df['misattribution'] == 1) & ~ignore_mask & ~ambiguity_mask).sum()

In [ ]:
extends_or_use_mask = (df['classification'] == 'extends') | (df['classification'] == 'uses')

In [ ]:
prompt1 = "The following sentences as from an academic paper, and cites a foundation model indicated by these <cite> citation brackets </cite>. These sentences have been identified as explicitly using the model, either by fine-tuning or using it out of the box, but we'd like to identify whether the authors were the ones doing so. Pretend you are the authors of the paper from which these sentences are pulled. Based on the following sentences, respond in the JSON format (without using ```json```) as {{\"response\": bool}} whether you were the one who used the model (true) or whether someone else did (false). Give one response for the whole set of sentences, without explanation. The sentence is as follows: {sentence}"

n = 5
model = partial(openAIReponse, model = 'gpt-4o-mini')
#model = ollamaResponse
def correct_attribution(sentence, n = n) -> List[bool]:
    responses = [model(prompt1.format(sentence = sentence)).lower() for i in range(n)]
    models = map(Response.model_validate_json, responses)
    total = np.array([model.response for model in models]).sum()
    return total/n 

def try_correct_attribution(*args, **kwargs):
    try:
        return correct_attribution(*args, **kwargs)
    except Exception as e:
        print(e)
        return None

column = f'gpt4omini_prompt1_{n}'
df[column] = mask_apply(df['multisentence'], extends_or_use_mask & ~ignore_mask, try_correct_attribution, fill_none = False)
df.to_csv(path, index=False)

In [ ]:
model_name = 'gpt-4o-mini'
cost_per_token = .15/1e6
token_counter = partial(num_tokens_from_string, model_name = model_name)

df['token_count'] = df['multisentence'].apply(lambda s: token_counter(prompt1.format(sentence = s)))
df['token_count'][extends_or_use_mask & ~ignore_mask].sum() * cost_per_token

In [ ]:
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/data/premicrosoft_meeting_classified.csv'
big_df = pd.read_csv(path)
extends_mask = big_df['classification'].apply(lambda s: s in {'extends', 'uses'})
ignore_mask = (big_df['multisentence'].apply(lambda s: s.find('\\begin{tab') != -1)) | (big_df['strippedModelKey'] == 'ase')
big_df_postprocess = big_df[extends_mask & ~ignore_mask]
big_df_postprocess['token_count'] = big_df_postprocess['multisentence'].apply(lambda s: token_counter(prompt1.format(sentence = s)))

In [ ]:
big_df_postprocess['token_count'].sum() * cost_per_token * 5

In [ ]:
df_filtered = df[df['misattribution'].notna() & df[column].notna()  & ~ignore_mask & context_fp]
cm = confusion_matrix(1-df_filtered['misattribution'].astype(int), (df_filtered[column] >= .8).astype(int))
labels = ['Incorrect', 'Correct']
sns.heatmap(cm, annot=True, fmt='d', cmap=ListedColormap(['lightgrey']),linewidths=0.5, xticklabels=labels, yticklabels=labels, cbar = False,annot_kws={"size": 30})
plt.ylabel('Manual')
plt.xlabel(column)

In [ ]:
((df_filtered['misattribution'].astype(int)) & (df_filtered[column] > .5).astype(int)).sum()

In [ ]:
postprocessing_promp1 = "You are a research scientist working on artificial intelligence. You wrote the following sentences in your paper, where you cite a foundation model indicated by these <cite> citation brackets </cite>. For each set of sentences, you will respond with a JSON format that determines whether you used the model, fine-tuned it, or only referenced others' work. The response will contain a field \"citation\" which is either \"used\" if you used the model directly, \"fine_tuned\" if you fine-tuned the model or a \"referenced\" if you only referenced others' work. Here is the sentence: {sentence}"
postprocessing_prompt2 = "You are a research scientist working on artificial intelligence, and you wrote the following sentences in a research paper. In these sentences, you reference a foundation model, indicated by these <cite></cite> citation brackets. For sentences provided, your task is to respond in JSON format, that should include a field called \"citation\" with one of the following values: \"uses\" – if you directly used, tested or run evaluation on the cited model in your research. \"extends\" – if you extended, fine-tuned, adapted or modified the cited model for your research. \"background\" – if you only referenced others' work that involved the cited model, without directly using or modifying it. Please analyze the following sentences accordingly: {sentence}"

model = partial(openAIReponse, model = 'gpt-4o-mini')
df['postprocess_reclass'] = mask_apply(df['multisentence'], None, lambda s: model(postprocessing_prompt2.format(sentence = s)), fill_none=False)

In [ ]:
threshold = .9

df['corrected_classification'] = mask_apply(df['classification'], (df['correct_attribution'] >= threshold) & (df['correct_attribution'].notna()), lambda s: 'context')

cm = confusion_matrix(df['alex'], df['corrected_classification'])
uses_fp = 1 - (cm[2,2]/(cm[:, 2].sum()))
extends_fp = 1 - (cm[1,1]/(cm[:, 1].sum()))

print(uses_fp, extends_fp)
cm

In [ ]:
def try_json(s):
    try:
        return Citation.model_validate_json(s).citation
    except Exception as e:
        print(e)
        return None
df['ollama']  = df['ollama_raw_response'].apply(lambda s: try_json(s))

In [ ]:
df.to_csv(path, index = False)